In [2]:

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf  # Version 1.0.0 (some previous versions are used in past commits)
from sklearn import metrics
import random
from random import randint
import time
import os
import numpy as np
from collections import namedtuple
import cv2
from pathlib import Path
import argparse
import os
from openvino.inference_engine import IENetwork, IECore
import csv
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow import keras

In [3]:
base_path = "H:\\이상행동"
folders = os.listdir(base_path)
file_name_lst = []
folder_path_lst = []
for folder in folders:
    folder_path = os.path.join(base_path, folder)
    folder_path_lst.append(folder_path)
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.mp4'):
            file_name_lst.append(os.path.join(folder_path, file_name))

In [4]:
len(file_name_lst)

1323

In [ ]:
folder_path_lst

In [ ]:
len(file_name_lst)

In [ ]:
file_name_lst[]

In [ ]:
folder_path_1 = "./data/이상(방화)"
folder_path_2 = "./data/이상(유기)"
folder_path_3 = "./data/이상(전도)"
folder_path_4 = "./data/이상(절도)"
folder_path_5 = "./data/이상(파손)"
folder_path_6 = "./data/이상(폭행)"
folder_path_7 = "./data/이상(흡연)"

In [ ]:
def concat(path):
    file_lst = os.listdir(path)
    df_list = []
    for file in file_lst:
        file_name = os.path.join(path, file)
        df = pd.read_csv(file_name)
        df = df[df.NUMOFBODIES != 0].reset_index(drop=True)
### 최대 인덱스 확인하고 싶으면 빼기
        if len(df) < 484: # 최대 인덱스 설정하기
            pad_length = 484 - len(df)
            pad_df = pd.DataFrame(0, index=range(len(df), 484), columns=df.columns)
            df = pd.concat([df, pad_df])
        df['index_num'] = df.index
        df_list.append(df)
###
    #concat_df = pd.concat(df_list, ignore_index=True)
    #    df_list.append(df)
    return df_list

In [ ]:
fire_mer = concat(folder_path_1)
yugi_mer = concat(folder_path_2)
jeon_mer = concat(folder_path_3)
theft_mer = concat(folder_path_4)
damage_mer = concat(folder_path_5)
violence_mer = concat(folder_path_6)
smoke_mer = concat(folder_path_7)

mer_lst = [fire_mer,
            yugi_mer,
            jeon_mer,
            theft_mer,
            damage_mer,
            violence_mer,
            smoke_mer]

In [ ]:
def lengs(name):
    lengths = []
    for i in range(len(name)):
        lengths.append(len(name[i]))
    return lengths

In [ ]:
lengs(fire_mer)

In [ ]:
lengsed = []
for i in mer_lst:
    lengsed.append(max(lengs(i)))
lengsed

In [ ]:
fire_mer = pd.concat(fire_mer, ignore_index=True)
yugi_mer = pd.concat(yugi_mer, ignore_index=True)
jeon_mer = pd.concat(jeon_mer, ignore_index=True)
theft_mer = pd.concat(theft_mer, ignore_index=True)
damage_mer = pd.concat(damage_mer, ignore_index=True)
violence_mer = pd.concat(violence_mer, ignore_index=True)
smoke_mer = pd.concat(smoke_mer, ignore_index=True)

In [ ]:
fire_mer['LABEL'] = 0  
yugi_mer['LABEL'] = 1  
jeon_mer['LABEL'] = 2
theft_mer['LABEL'] = 3
damage_mer['LABEL'] = 4
violence_mer['LABEL'] = 5
smoke_mer['LABEL'] = 6

In [ ]:
mer_lst = [fire_mer,
            yugi_mer,
            jeon_mer,
            theft_mer,
            damage_mer,
            violence_mer,
            smoke_mer]
full_data = pd.concat(mer_lst, ignore_index=True)
full_data.shape

In [ ]:
# 데이터와 레이블을 추출합니다.
data_without_label = full_data.drop(['ID','LABEL', 'TIMESTAMP', 'FRAME_NUM','PELVIS_CONFIDENCE_LEVEL', 'SPINE_NAVA_CONFIDENCE_LEVEL', 'SPINE_CHEST_CONFIDENCE_LEVEL', 'NECK_CONFIDENCE_LEVEL', 
                                     'SHOULDER_LEFT_CONFIDENCE_LEVEL', 'ELBOW_LEFT_CONFIDENCE_LEVEL', 'WRIST_LEFT_CONFIDENCE_LEVEL', 'SHOULDER_RIGHT_CONFIDENCE_LEVEL', 
                                     'ELBOW_RIGHT_CONFIDENCE_LEVEL', 'WRIST_RIGHT_CONFIDENCE_LEVEL', 'HIP_LEFT_CONFIDENCE_LEVEL', 'KNEE_LEFT_CONFIDENCE_LEVEL', 'ANKLE_LEFT_CONFIDENCE_LEVEL',
                                     'HIP_RIGHT_CONFIDENCE_LEVEL', 'KNEE_RIGHT_CONFIDENCE_LEVEL', 'ANKLE_RIGHT_CONFIDENCE_LEVEL', 'HEAD_CONFIDENCE_LEVEL'], axis=1)
label_data = full_data['LABEL']

In [ ]:
sequence_start_indices = full_data.index[full_data['index_num'] == 0].tolist()
sequence_lengths = [sequence_start_indices[i] - sequence_start_indices[i-1] for i in range(1, len(sequence_start_indices))]
sequence_lengths.insert(0, sequence_start_indices[0])

# Min sequence length
min_sequence_length = min(sequence_lengths)

In [ ]:
min_sequence_length = 484

In [ ]:
X_sequences = []
y_sequences = []

# Iterating over the start indices to create sequences
for i in range(len(sequence_start_indices) - 1):
    start_index = sequence_start_indices[i]
    end_index = sequence_start_indices[i + 1]
    
    # Ensure the sequence has the expected length before appending
    if end_index - start_index == min_sequence_length:
        X_sequence = data_without_label.iloc[start_index:end_index].values
        y_sequence = label_data.iloc[start_index:end_index].values
        X_sequences.append(X_sequence)
        y_sequences.append(y_sequence[-1])  # Taking the label of the last frame for the sequence

# Converting lists to numpy arrays
X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)

X_sequences.shape, y_sequences.shape


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(X_sequences, y_sequences, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)

In [ ]:
print(X_train.shape, y_train.shape)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# 모델 정의
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(7, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 훈련 데이터에 대한 샘플 가중치 마스크 생성
# 패딩이 아닌 값은 1, 패딩된 값은 0
sample_weight_train = np.where(X_train != 0, 1, 0)
sample_weight_train = sample_weight_train.max(axis=-1)

# 각 시퀀스에 대한 평균 샘플 가중치 계산
sample_weight_train_avg = sample_weight_train.mean(axis=1)

In [ ]:
model_file_path = "./model/notnormal_lstm_model1.h5"
checkpoint_cb = keras.callbacks.ModelCheckpoint(model_file_path)
# - 자동 훈련 멈추기 함수 사용 : 추가 훈련 epoch 2회, 가중치 업데이트
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                        restore_best_weights=True)


# 4. Model Training
# 모델 훈련
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=1000,
                    batch_size=128, sample_weight=sample_weight_train_avg,
                    callbacks=[checkpoint_cb, early_stopping_cb])

# 5. Model Evaluation
# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy*100:.2f}%")


In [ ]:
from keras.models import load_model
model1 = load_model('./model/best_lstm_model1.h5')

In [ ]:
model1.summary()

In [ ]:
### 불러온 모델로 데이터 정확도 확인하기
# - 예측
test_preds = model1.predict(X_val)
test_preds.shape

In [ ]:

# - 정답갯수 / 오답 갯수 (100개 데이터에 대해서만)
# - 정답률 / 오답률 (100개 데이터에 대해서만)
test_pred_idx = [np.argmax(pred) for pred in test_preds]
test_pred_idx

all_cnt = len(test_pred_idx)
o_cnt = len(y_val[(test_pred_idx == y_val)])
x_cnt = len(y_val[(test_pred_idx != y_val)])

print("정답갯수 : {}개, 오답갯수 : {}개".format(o_cnt, x_cnt))
print("정답률 : {}%, 오답률 : {}%".format(o_cnt/all_cnt*100,
                                        x_cnt/all_cnt*100))

In [ ]:
cate = ['이상(방화)',
'이상(유기)',
'이상(전도)',
'이상(절도)',
'이상(파손)',
'이상(폭행)',
'이상(흡연)',]

In [ ]:
# 유니크한 종속변수들을 얻습니다.
unique_targets = np.unique(y_val)

# 테스트 데이터에 대한 예측을 한 번만 수행
test_preds = model1.predict(X_val)
test_pred_idx = [np.argmax(pred) for pred in test_preds]

In [ ]:
np.unique(test_pred_idx)

In [ ]:
test_pred_idx = np.array(test_pred_idx)

In [ ]:
# 각 종속변수별 정확도를 저장할 데이터프레임
accuracy_per_target = pd.DataFrame(index=unique_targets, columns=['accuracy'])

In [ ]:
# 각 클래스별 정확도 계산
class_accuracies = {}
for i in np.unique(y_val):
    idx = np.where(y_val == i)
    correct = np.sum(test_pred_idx[idx] == y_val[idx])
    total = len(idx[0])
    class_accuracies[i] = correct / total
    accuracy_per_target.loc[i, 'accuracy'] = class_accuracies[i]

In [ ]:
accuracy_per_target

In [ ]:
accuracy_per_target["Target"] = cate

In [ ]:
accuracy_df_sorted = accuracy_per_target.sort_values(by='accuracy', ascending=True)

In [ ]:
accuracy_df_sorted.head(20)